In [13]:
import tensorflow as tf
import bert
import tensorflow_hub as hub

In [14]:
# model = tf.keras.models.load_model('models/text_model_1')
# model.summary()

In [15]:
#make pre-train for bert
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [16]:
def tokenize_data(data):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(data))

In [17]:
s = """
    I am trying to create a multi label text classification model in tensorflow. I was not sure if I should use
    Tensorflow or PyTorch, I decided to use Tensorflow because I am more familiar with it. I am testing this model
    with the freecode dataset and TagBERT. Is this the best model for my use case?
    """

s2 = """
    I am building a Javascript website which will render 3D graphics. It is visualizing different mechanical
    components of airplanes. What software do you recommend to use for this purpose, are there any good Javascript
    libraries I can use for this?
    """

the_tokens = tokenize_data(s2)
the_tokens

[1045,
 2572,
 2311,
 1037,
 9262,
 22483,
 4037,
 2029,
 2097,
 17552,
 7605,
 8389,
 1012,
 2009,
 2003,
 5107,
 6026,
 2367,
 6228,
 6177,
 1997,
 24042,
 1012,
 2054,
 4007,
 2079,
 2017,
 16755,
 2000,
 2224,
 2005,
 2023,
 3800,
 1010,
 2024,
 2045,
 2151,
 2204,
 9262,
 22483,
 8860,
 1045,
 2064,
 2224,
 2005,
 2023,
 1029]

In [18]:
PREDICTION_BATCH_SIZE = 512
chunk = [the_tokens]
processed_chunk_dataset = tf.data.Dataset.from_generator(lambda: chunk, output_types=(tf.int32))
batched_chunk_dataset = processed_chunk_dataset.padded_batch(PREDICTION_BATCH_SIZE, padded_shapes=(None,))

# rows = text_model.predict(batched_chunk_dataset, batch_size=len(chunk))
# model.predict(batched_chunk_dataset)

In [19]:
import json
with open("labels.json", 'r') as jsonfile:
    labels_json = json.load(jsonfile)

labels_json

{'0': 'Systems-Administration',
 '1': 'Information-Management',
 '2': 'Issue-Tracking',
 '3': 'PHP',
 '4': 'php-classes',
 '5': 'Security',
 '6': 'LDAP',
 '7': 'SQL',
 '8': 'Operating-Systems',
 '9': 'Monitoring',
 '10': 'Software-Development',
 '11': 'Internet',
 '12': 'FTP',
 '13': 'Mirroring',
 '14': 'Web',
 '15': 'Web-Application',
 '16': 'Communications',
 '17': 'Telephony',
 '18': 'Office/Business',
 '19': 'Point-Of-Sale',
 '20': 'Game',
 '21': 'Database',
 '22': 'MySQL',
 '23': 'Desktop-Environment',
 '24': 'Terminals',
 '25': 'Terminal-Emulators/X-Terminals',
 '26': 'Site-Management',
 '27': 'Log-Analysis',
 '28': 'Libraries',
 '29': 'Benchmark',
 '30': 'Logging',
 '31': 'Networking',
 '32': 'Utilities',
 '33': 'PDF',
 '34': 'Java',
 '35': 'Embedded-Systems',
 '36': 'C++',
 '37': 'Library',
 '38': 'Interpreters',
 '39': 'Command-Line',
 '40': 'Scientific/Engineering',
 '41': 'Version-Control',
 '42': 'C',
 '43': 'Email',
 '44': 'Filters',
 '45': 'Dynamic-Content',
 '46': 'CGI-T

In [20]:
res_dict = {}

for i in labels_json.keys():
    model = tf.keras.models.load_model('models/text_model_group_1/text_model_{0}'.format(i))

    res = model.predict(batched_chunk_dataset)[0][0]
    
    res_dict.update({labels_json[i]: res})
    
    print("{0}: {1}".format(labels_json[i], res))

Systems-Administration: 2.0894198314636014e-05
Information-Management: 0.0013996448833495378
Issue-Tracking: 0.15374040603637695
PHP: 0.01917734369635582
php-classes: 0.09765300899744034
Security: 0.00022978676133789122
LDAP: 0.3083130121231079
SQL: 0.36477166414260864
Operating-Systems: 0.014629865065217018
Monitoring: 0.0020909819286316633
Software-Development: 0.9987195730209351
Internet: 1.242358536046595e-07
FTP: 0.12670668959617615
Mirroring: 0.3074641227722168
Web: 0.001299111288972199
Web-Application: 0.38046982884407043
Communications: 0.006402055732905865
Telephony: 0.26401230692863464
Office/Business: 0.00832776166498661
Point-Of-Sale: 0.4257253408432007
Game: 0.16612504422664642
Database: 1.1054736205551308e-05
MySQL: 0.09029294550418854
Desktop-Environment: 0.01452754158526659
Terminals: 0.111082062125206
Terminal-Emulators/X-Terminals: 0.2667604088783264
Site-Management: 0.0028197220526635647
Log-Analysis: 0.0391605906188488
Libraries: 0.9446112513542175
Benchmark: 0.4751

In [21]:
{k: v for k, v in res_dict.items() if v > 0.8}

{'Software-Development': 0.9987196,
 'Libraries': 0.94461125,
 'Scientific/Engineering': 0.96935195,
 'multimedia': 0.9960692,
 'Visualization': 0.99907494,
 'Graphics': 0.99960965,
 'User-Interfaces': 0.9314361,
 'JavaScript': 0.9757648,
 '3D-Rendering': 0.9996099,
 '3D-Modeling': 0.98921007}